In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import random
import pickle
import util
from copy import deepcopy
from genetic import *
from os.path import join as pjoin

pop_dir = pjoin('..', 'pop_data', 'tourn_8')
json_dir = pjoin('pop_json')

params = util.parse_json(pjoin(json_dir, 'baseline_random.json'))
params['mutation'] = 0.1
params['tour_size'] = 8
print '\n'.join('%s: %s' % (k,v) for k,v in params.iteritems())

# Check if it must use 'l' or 'r' sufix (local or random)
lr = 'r' if params['local'] is None else 'l'

# Create the format string containing the name of the data file
pic_name = pjoin(pop_dir, '%02d' + lr + '_100%s.dat')
print '\nPickled file name: ', pic_name

# Ports to communicate with R
ports = range(5000, 5002)
print '\nPorts', ports

In [ ]:
# SOME HELPER FUNCTIONS

def flatten_tuple_list(x, size = None):
    """
    Transform a list of tuples in a list containing all tuples' elements, flattening it.
    
    Example:
        >>> flatten_tuple_list([(1,2,3), (1,), (4,5)])
        [1, 2, 3, 1, 4, 5]
        
        >>> flatten_tuple_list([(1,2,3), (1,), (4,5)], 3)
        [1, 2, 3]
    
    Parameters
    ----------
    x : list
        A list containing tuples to be flattened
    size : int
        Size of the list to be returned

    Returns
    -------
    list
        A list, with size `size` containing the elements located in the tuples received.
    """
    return [el for tup in x for el in tup][:size]

def communicate_and_evaluate():
    for sock in Chromo.sockets:
        util.send_msg(sock, 'NEWGEN\n')
    best = pop.evaluate()
    for sock in Chromo.sockets:
        util.send_msg(sock, 'ENDGEN\n')
    print 'main:current_best ', best
    return best

def selection():
    # Select parents and flatten to dump into a file
    par_list = []
    par_fitness = []
    while 2*len(par_list) < pop.size:
        # Get the index of the selected parents
        p1 = pop.tournament_selection()
        p2 = pop.tournament_selection()

        # Save the fitness in a flattened list
        par_fitness.append(pop.fitness[p1])
        par_fitness.append(pop.fitness[p2])

        # Save parents' chromosomes in a list of tuples
        parents = deepcopy((pop.population[p1],
                           pop.population[p2]))
        par_list.append(parents)
    return par_list, par_fitness

def crossover(par_list):
    child_list = []
    for parents in par_list:
        childs = ()
        if random.random() < pop.crossover:
            print 'Y',
            childs = Chromo.crossover(parents)
        else:
            print 'N',
            childs = parents
        print
        child_list.append(childs)
    return child_list

def mutation(child_list):
    mut_list = []
    for childs in child_list:
        for ch in childs:
            if random.random() < pop.mutation:
                print '\tY',
                ch = Chromo.mutate(ch)
            else:
                print '\tN',
            mut_list.append(ch)
    print
    return mut_list

# 1st generation of population -  Creation

Setting the parameters of the population.

After this there are 2 options:
1. Evaluate the population and store its fitness
2. Load an already evaluated population

In [ ]:
# Setting population parameters
global pop
pop = Population( size = params['size'],
        crossover = params['crossover'],
        mutation = params['mutation'],
        elitism = params['elitism'],
        imigration = params['imigration'],
        tour_size = params['tour_size'],
        local = params['local'])

# Choose crossover operator
Chromo.cross_op = params['cross_op']

# Initialize generation counter
gen = 1


# Open sockets to communicate with R
sockets = []
for port in ports:
    sock = util.open_socket(int(port))
    sockets.append(sock)
Chromo.sockets = sockets

### 1 - Evaluating the population

In [ ]:
# Signaling R and evaluating
best = communicate_and_evaluate()

# Saving the population and its fitness
with open(pic_name % (gen, ''), 'wb') as pic_file:
    pickle.dump(pop.population, pic_file)
    pickle.dump(pop.fitness, pic_file)

### 2 - Loading the population

In [ ]:
# Loading the population and its fitness
with open(pic_name % (gen, ''), 'rb') as pic_file:
    pop.population = pickle.load(pic_file)
    pop.fitness = pickle.load(pic_file)
original_pop = deepcopy(pop.population)

# Genetic operators - Selection

Steps for evolution:
1. **Selection**
2. Crossover
3. Mutation

In [ ]:
# Loading the population and its fitness
with open(pic_name % (gen, ''), 'rb') as pic_file:
    pop.population = pickle.load(pic_file)
    pop.fitness = pickle.load(pic_file)

par_list, par_fitness = selection()

# Saving the population and its fitness
with open(pic_name % (gen, '_sel'), 'wb') as pic_file:
    pickle.dump(par_list, pic_file)
    pickle.dump(par_fitness, pic_file)

# Genetic operators -  Crossover

Steps for evolution:
1. Selection
2. **Crossover**
3. Mutation

In [ ]:
# Loading the population and its fitness
with open(pic_name % (gen, '_sel'), 'rb') as pic_file:
    par_list = pickle.load(pic_file)

# Crossover
child_list = crossover(par_list)

# Flatten population and evaluate
pop.population = deepcopy(flatten_tuple_list(child_list, pop.size))
best = communicate_and_evaluate()

# Saving the childs and their fitness
with open(pic_name % (gen, '_cross'), 'wb') as pic_file:
    pickle.dump(child_list, pic_file)
    pickle.dump(pop.fitness, pic_file)

# Genetic operators -  Mutation
Also creation of the next generation

Steps for evolution:
1. Selection
2. Crossover
3. **Mutation**

In [ ]:
# Loading the population and its fitness
with open(pic_name % (gen, '_cross'), 'rb') as pic_file:
    child_list = pickle.load(pic_file)

# Mutate and evaluate
pop.population = mutation(child_list)
best = communicate_and_evaluate()


####################################
# PASSING TO NEXT GENERATION
gen += 1

# Saving the population and its fitness
with open(pic_name % (gen, ''), 'wb') as pic_file:
    pickle.dump(pop.population, pic_file)
    pickle.dump(pop.fitness, pic_file)

# Genetic operations over 2nd generation

Here the same genetic operations seen before are applied to the 2nd generations.

Te goal here is to save only the selected parents and then get to the 3rd generation. However, if you want to save all operations until the 3rd gen you only have to run the 3 code cells above (containing the operations).

In [ ]:
# Loading the population and its fitness
with open(pic_name % (gen, ''), 'rb') as pic_file:
    pop.population = pickle.load(pic_file)
    pop.fitness = pickle.load(pic_file)

par_list, par_fitness = selection()

# Saving the population and its fitness
with open(pic_name % (gen, '_sel'), 'wb') as pic_file:
    pickle.dump(par_list, pic_file)
    pickle.dump(par_fitness, pic_file)

####################################

# Crossover
child_list = crossover(par_list)

# Mutate and evaluate
pop.population = mutation(child_list)
best = communicate_and_evaluate()

####################################
# PASSING TO NEXT GENERATION
gen += 1

# Saving 10th, 20th and 30th generations

In [ ]:
for i in range(gen, 30):
    if i%10 == 0 or i == gen:
        # Loading the population and its fitness (saved previously)
        with open(pic_name % (i, ''), 'rb') as pic_file:
            pop.population = pickle.load(pic_file)
    
    print 'GENERATION', i
    pop.evolve()
    best = communicate_and_evaluate()

    if (i+1)%10 == 0:
        # Saving the population and its fitness
        with open(pic_name % (i+1, ''), 'wb') as pic_file:
            pickle.dump(pop.population, pic_file)
            pickle.dump(pop.fitness, pic_file)

In [ ]:
print gen
gen = 6
print gen